# Walmart Products Dataset Preprocessing

This script performs preliminary cleaning of the Walmart Products dataset from Kaggle, which can be found [here](https://www.kaggle.com/datasets/thedevastator/product-prices-and-sizes-from-walmart-grocery/code).

Some of the cleaning steps were inspired by a similar project on Walmart price prediction on Kaggle, which can be found [here](https://www.kaggle.com/code/ryanbell62101/walmart-product-price-predictor).



In [5]:
# Necessary Imports
import numpy as np
import pandas as pd
import re

## Step 1: Read in Dataset

First, we want to a look at the dataset and verify that it imported successfully.

In [6]:
# Read in dataset and display the top couple of rows to verify it imported properly
walmart = pd.read_csv('walmart_dataset.csv')
walmart.head()

/var/folders/7p/xb2glmy50v59_g4kfskbtjz80000gn/T/ipykernel_13353/345100623.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  walmart = pd.read_csv('walmart_dataset.csv')


,index,SHIPPING_LOCATION,DEPARTMENT,CATEGORY,SUBCATEGORY,BREADCRUMBS,SKU,PRODUCT_URL,PRODUCT_NAME,BRAND,PRICE_RETAIL,PRICE_CURRENT,PRODUCT_SIZE,PROMOTION,RunDate,tid
0,0,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",110895339,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Red Pepper Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163804
1,1,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",105455228,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Garlic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163805
2,2,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",128642379,https://www.walmart.com/ip/Marketside-Classic-...,"Marketside Classic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163806
3,3,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",366126367,https://www.walmart.com/ip/Marketside-Everythi...,"Marketside Everything Hummus, 10 oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163807
4,4,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",160090316,https://www.walmart.com/ip/Price-s-Jalapeno-Di...,"Price's Jalapeno Dip, 12 Oz.",Price's,3.12,3.12,12,NaN,2022-09-11 21:20:04,16163808


## Step 2: Look for Unique Values in each Feature

Each feature that provides meaningful information should have a decent amount of unique values across a dataset with 569k entries. Therefore, we want to remove any features that have low numbers of unique values, as low uniqueness indicates there is not much variation in that feature across the data entries.

In [7]:
# Analyze number of unique items in each column
unique = walmart.nunique()
unique

index                568534
SHIPPING_LOCATION        26
DEPARTMENT               14
CATEGORY                114
SUBCATEGORY             125
BREADCRUMBS             116
SKU                   30827
PRODUCT_URL           32008
PRODUCT_NAME          30688
BRAND                  4368
PRICE_RETAIL           1852
PRICE_CURRENT          1833
PRODUCT_SIZE           1290
PROMOTION                 0
RunDate                   1
tid                  568534
dtype: int64

In [8]:
# Drop "Promotion" and "RunDate" features as they only have 0 and 1 unique values respectively, so they provide no valuable info
walmart.drop(columns=['RunDate', 'PROMOTION'], inplace=True)


## Step 3: Deal with Missing Data

Next, we want to analyze how much missing data (typically "NA" values) are in the dataset.

In [9]:
# sum all missing values for each feature
walmart.isna().sum()

index                     0
SHIPPING_LOCATION         0
DEPARTMENT                0
CATEGORY                  0
SUBCATEGORY          207210
BREADCRUMBS               0
SKU                       0
PRODUCT_URL               0
PRODUCT_NAME              0
BRAND                    27
PRICE_RETAIL              0
PRICE_CURRENT             0
PRODUCT_SIZE          62825
tid                       0
dtype: int64

Here, we see that Brand and Product Size have a relatively minor amount of NA values, in comparison to Subcategory which has 207k NA values. Therefore, we want to initially keep the entries with NA values for Subcategory to avoid removing too much data. In the meantime, we choose to get rid of the entries with NA for Brand and Product Size for now.

In [10]:
# Subcategory has too many missing values to remove all associated data, otherwise dataset would significantly shrink
walmart['SUBCATEGORY'].fillna('none', inplace=True)

walmart.dropna(inplace=True) # Drop entries with NA values for product_name and product_size
walmart.isna().sum()

/var/folders/7p/xb2glmy50v59_g4kfskbtjz80000gn/T/ipykernel_13353/2297702649.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  walmart['SUBCATEGORY'].fillna('none', inplace=True)


index                0
SHIPPING_LOCATION    0
DEPARTMENT           0
CATEGORY             0
SUBCATEGORY          0
BREADCRUMBS          0
SKU                  0
PRODUCT_URL          0
PRODUCT_NAME         0
BRAND                0
PRICE_RETAIL         0
PRICE_CURRENT        0
PRODUCT_SIZE         0
tid                  0
dtype: int64

### Step 3: Analyze Data Types for Each Feature

Now, we should verify the data types for each feature to ensure that each data type makes logical sense and will be what we want to use moving forward.

In [11]:
# Look at overview of dataset thus far
walmart.info()

<class 'pandas.core.frame.DataFrame'>
Index: 505709 entries, 0 to 568533
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              505709 non-null  int64  
 1   SHIPPING_LOCATION  505709 non-null  int64  
 2   DEPARTMENT         505709 non-null  object 
 3   CATEGORY           505709 non-null  object 
 4   SUBCATEGORY        505709 non-null  object 
 5   BREADCRUMBS        505709 non-null  object 
 6   SKU                505709 non-null  int64  
 7   PRODUCT_URL        505709 non-null  object 
 8   PRODUCT_NAME       505709 non-null  object 
 9   BRAND              505709 non-null  object 
 10  PRICE_RETAIL       505709 non-null  float64
 11  PRICE_CURRENT      505709 non-null  float64
 12  PRODUCT_SIZE       505709 non-null  object 
 13  tid                505709 non-null  int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 57.9+ MB


Here, we notice that product size is an "object" instead of a number. This should be a numerical value instead of a string.

In [12]:
# Product size should be numerical, take digits out of string and convert into numerical format
def get_digits(string):
    digit_search = re.search('([0-9]+)', string)
    return digit_search.group(1) if digit_search else None

walmart['PRODUCT_SIZE'] = pd.to_numeric(walmart['PRODUCT_SIZE'].map(get_digits))

Another issue is that, for entries with strings such as Product Category, Product Subcategory, Brand, and Breadcrumbs, we should standardize letter casing in order to remove duplicate entries.

In [13]:
# Eliminate duplicate entries by converting all text to lowercase
walmart['CATEGORY'] = walmart['CATEGORY'].str.lower()
walmart['SUBCATEGORY'] = walmart['SUBCATEGORY'].str.lower()
walmart['BRAND'] = walmart['BRAND'].str.lower()
walmart['BREADCRUMBS'] = walmart['BREADCRUMBS'].str.lower()

walmart.nunique()

index                505709
SHIPPING_LOCATION        26
DEPARTMENT               14
CATEGORY                113
SUBCATEGORY             121
BREADCRUMBS             115
SKU                   26604
PRODUCT_URL           27634
PRODUCT_NAME          26537
BRAND                  3871
PRICE_RETAIL           1709
PRICE_CURRENT          1684
PRODUCT_SIZE            137
tid                  505709
dtype: int64

## Step 4: Final Clean and Organization of Data

In [14]:
# Check if any new NA values were introduced in the above steps
walmart.isna().sum()

index                  0
SHIPPING_LOCATION      0
DEPARTMENT             0
CATEGORY               0
SUBCATEGORY            0
BREADCRUMBS            0
SKU                    0
PRODUCT_URL            0
PRODUCT_NAME           0
BRAND                  0
PRICE_RETAIL           0
PRICE_CURRENT          0
PRODUCT_SIZE         202
tid                    0
dtype: int64

In [15]:
# Drop any new NA values and re-verify
walmart.dropna(inplace=True)
walmart.isna().sum()

index                0
SHIPPING_LOCATION    0
DEPARTMENT           0
CATEGORY             0
SUBCATEGORY          0
BREADCRUMBS          0
SKU                  0
PRODUCT_URL          0
PRODUCT_NAME         0
BRAND                0
PRICE_RETAIL         0
PRICE_CURRENT        0
PRODUCT_SIZE         0
tid                  0
dtype: int64

In [16]:
# Sort the data in a hierarchical structure by categories followed by subcategories
category_groups = walmart.groupby(['CATEGORY','SUBCATEGORY'])
category_groups[['PRICE_RETAIL']].describe()

PRICE_RETAIL                       \
                                               count       mean       std   
CATEGORY                 SUBCATEGORY                                        
bacon, hot dogs, sausage none                 5939.0   5.474102  3.054611   
baking nuts & seeds      none                  552.0   6.776793  4.086878   
baking soda & starch     none                  434.0   3.898710  3.824149   
beef jerky               none                 1510.0   8.375377  3.907909   
beer                     domestic beer        1490.0  12.444389  5.758637   
...                                              ...        ...       ...   
wine                     sparkling wine        739.0  13.576685  8.817995   
                         specialty wine         48.0   9.535417  3.102972   
                         white wine           2190.0  10.602868  4.910865   
yeast                    none                  237.0   3.521561  1.981249   
yogurt                   none                 6773.0   2.897639  2.017740   

                                                                          
                                          min   25%    50%    75%    max  
CATEGORY                 SUBCATEGORY                                      
bacon, hot dogs, sausage none            0.84  3.78   4.72   6.84  24.66  
baking nuts & seeds      none            1.18  3.24   6.12   9.30  17.92  
baking soda & starch     none            0.72  1.48   2.48   4.12  15.86  
beef jerky               none            1.08  4.98   7.88  11.98  18.58  
beer                     domestic beer   1.48  7.99  12.98  16.98  27.98  
...                                       ...   ...    ...    ...    ...  
wine                     sparkling wine  3.72  8.98  11.48  14.98  67.27  
                         specialty wine  5.48  6.98   8.98  11.48  18.98  
                         white wine      2.96  6.99   9.98  12.98  90.00  
yeast                    none            0.86  1.72   4.62   5.18   6.37  
yogurt                   none            0.52  1.22   2.18   4.37  15.18  

[208 rows x 8 columns]

In [17]:
# Save cleaned dataset
walmart.to_csv("walmart_cleaned.csv", index=False)

In [ ]:
import pandas as pd

filename = "walmart_cleaned.csv"
column_to_check = "SHIPPING_LOCATION"

print(f"Loading file: {filename}...")
df = pd.read_csv(filename, usecols=[column_to_check])

print(f"\n--- Details for column '{column_to_check}' ---")

missing_values = df[column_to_check].isna().sum()
print(f"\n1. Missing values (NA) count: {missing_values}")

unique_count = df[column_to_check].nunique()
print(f"\n2. Number of unique store IDs: {unique_count}")

unique_ids = df[column_to_check].unique()
print("\n3. List of unique store IDs:")
print(unique_ids)

print("\n4. Value counts per ID:")
print(df[column_to_check].value_counts())
    

Loading file: walmart_cleaned.csv...

--- Details for column 'SHIPPING_LOCATION' ---

1. Missing values (NA) count: 0

2. Number of unique store IDs: 26

3. List of unique store IDs:
[79936 48180 96797 94565 23223 75211 89108 15601 23111 33012 72034 60007
 63376 44035 53215 33647  6010 66062 90650 78130 85225 75052 70072 77449
 30044 45011]

4. Value counts per ID:
SHIPPING_LOCATION
33647    23526
23223    23085
44035    23067
77449    22370
75052    22367
48180    22254
15601    22233
53215    22150
70072    21964
23111    21885
60007    21833
45011    21802
75211    21622
78130    21412
85225    20144
30044    19887
90650    19469
66062    19161
63376    19141
72034    18176
89108    17887
79936    17555
33012    15371
96797     9917
94565     8783
6010      8446
Name: count, dtype: int64

--- Details for column 'SHIPPING_LOCATION' ---

1. Missing values (NA) count: 0

2. Number of unique store IDs: 26

3. List of unique store IDs:
[79936 48180 96797 94565 23223 75211 89108 15601 231

In [ ]:
import pandas as pd

cleaned_walmart_file = "walmart_cleaned.csv"
zip_database_file = "uszips.csv" 
print(f"Loading cleaned Walmart data: {cleaned_walmart_file}...")
walmart_df = pd.read_csv(cleaned_walmart_file, dtype=str)

print(f"Loading ZIP database: {zip_database_file}...")
zip_db = pd.read_csv(zip_database_file, dtype=str)

# --- Key preparation: standardize column names and types ---
# Support common name variants: 'zip' + ('lat','lng') or 'zip_code' + ('latitude','longitude')
if 'zip' in zip_db.columns:
    zip_db.rename(columns={
        'zip': 'SHIPPING_LOCATION',
        'lat': 'latitude',
        'lng': 'longitude'
    }, inplace=True)
elif 'zip_code' in zip_db.columns:
    zip_db.rename(columns={'zip_code': 'SHIPPING_LOCATION'}, inplace=True)
else:
    print("--- Error: could not find 'zip' or 'zip_code' column in ZIP database ---")
    print("Please inspect the CSV and update the rename logic accordingly.")
    raise KeyError("Zip code column not found")

zip_db['SHIPPING_LOCATION'] = zip_db['SHIPPING_LOCATION'].astype(str).str.strip().str.zfill(5)

for geo_col in ('latitude', 'longitude'):
    if geo_col in zip_db.columns:
        zip_db[geo_col] = pd.to_numeric(zip_db[geo_col], errors='coerce')

columns_to_merge = ['SHIPPING_LOCATION', 'latitude', 'longitude', 'city', 'state_id']
columns_to_merge = [col for col in columns_to_merge if col in zip_db.columns]

print("Merging coordinates into Walmart data...")
walmart_with_coords = pd.merge(
    walmart_df,
    zip_db[columns_to_merge],  # only merge the columns we need
    on="SHIPPING_LOCATION",   # merge key (e.g., '00601', '30044')
    how="left"                # keep all Walmart items even if some ZIPs don't match
)

print("\n--- Merge complete! ---")
print("Preview of merged data (includes coordinates):")

preview_cols = [c for c in ['PRODUCT_NAME', 'SHIPPING_LOCATION', 'latitude', 'longitude', 'city'] if c in walmart_with_coords.columns]
print(walmart_with_coords[preview_cols].head())

unmatched_rows = walmart_with_coords['latitude'].isna().sum() if 'latitude' in walmart_with_coords.columns else 0
if unmatched_rows > 0:
    print(f"\n Warning: {unmatched_rows} rows did not match a coordinate.")
else:
    print("\n All rows matched coordinates!")

output_file = "walmart_with_coordinates.csv"
walmart_with_coords.to_csv(output_file, index=False)
print(f"\nSaved enriched dataset to: {output_file}")

Loading cleaned Walmart data: walmart_cleaned.csv...
Loading ZIP database: uszips.csv...
Merging coordinates into Walmart data...

--- Merge complete! ---
Preview of merged data (includes coordinates):
Loading ZIP database: uszips.csv...
Merging coordinates into Walmart data...

--- Merge complete! ---
Preview of merged data (includes coordinates):
                                  PRODUCT_NAME SHIPPING_LOCATION  latitude  \
0  Marketside Roasted Red Pepper Hummus, 10 Oz             79936  31.77373   
1      Marketside Roasted Garlic Hummus, 10 Oz             79936  31.77373   
2             Marketside Classic Hummus, 10 Oz             79936  31.77373   
3          Marketside Everything Hummus, 10 oz             79936  31.77373   
4                 Price's Jalapeno Dip, 12 Oz.             79936  31.77373   

   longitude     city  
0 -106.29631  El Paso  
1 -106.29631  El Paso  
2 -106.29631  El Paso  
3 -106.29631  El Paso  
4 -106.29631  El Paso  

                                  P

### Diagnosis and imputation strategies for unmatched ZIP codes

The following cell will:
- Show the distribution of `SHIPPING_LOCATION` values that failed to merge (sorted by frequency).
- Attempt imputation using the mean latitude/longitude for the ZIP prefix (first 3 digits).
- If available, try imputing using state-level centroids (`state_id`).
- Finally save the imputed dataset to `walmart_with_coordinates_imputed.csv` and report remaining unmatched counts.

These methods provide reasonable approximate locations for analysis and visualization without calling external geocoding APIs.

In [ ]:
import numpy as np
import pandas as pd
print('walmart_with_coords columns:', list(walmart_with_coords.columns))

# 1) Show examples and frequencies of unmatched SHIPPING_LOCATION values
unmatched_mask = walmart_with_coords['latitude'].isna()
unmatched = walmart_with_coords[unmatched_mask].copy()
total_unmatched = len(unmatched)
print(f'Total unmatched rows: {total_unmatched}')
if total_unmatched == 0:
    print('No unmatched rows — no imputation required.')
else:
    print('\nTop 30 unmatched SHIPPING_LOCATION values (with counts):')
    print(unmatched['SHIPPING_LOCATION'].value_counts().head(30))

    # 2) Try prefix-based imputation: build mean lat/lon per 3-digit prefix
    # Only possible if zip_db contains latitude/longitude
    if 'latitude' in zip_db.columns and 'longitude' in zip_db.columns:
        zip_db['prefix'] = zip_db['SHIPPING_LOCATION'].astype(str).str[:3]
        prefix_centroid = zip_db.groupby('prefix')[['latitude','longitude']].mean().reset_index().set_index('prefix')
        print(f'Number of prefixes available for prefix-based imputation: {len(prefix_centroid)}')
    else:
        prefix_centroid = None
        print('zip database lacks latitude/longitude — prefix imputation not available.')

    # 3) If state_id is present, build state-level centroids (preferable to global mean)
    state_centroid = None
    if 'state_id' in zip_db.columns and 'latitude' in zip_db.columns and 'longitude' in zip_db.columns:
        state_centroid = zip_db.groupby('state_id')[['latitude','longitude']].mean().reset_index().set_index('state_id')
        print(f'Number of states available for state-based imputation: {len(state_centroid)}')

    # 4) Imputation order: prefix -> state -> global mean
    global_mean = None
    if 'latitude' in zip_db.columns and 'longitude' in zip_db.columns:
        global_mean = zip_db[['latitude','longitude']].astype(float).mean().to_dict()


    def impute_row(row):

        if not pd.isna(row['latitude']):
            return row
        ship = str(row['SHIPPING_LOCATION']) if pd.notna(row['SHIPPING_LOCATION']) else ''
        prefix = ship[:3] if len(ship) >= 3 and ship[:3].isdigit() else None

        if prefix and prefix_centroid is not None and prefix in prefix_centroid.index:
            row['latitude'] = prefix_centroid.at[prefix,'latitude']
            row['longitude'] = prefix_centroid.at[prefix,'longitude']
            row['_imputed_by'] = f'prefix:{prefix}'
            return row

        if 'state_id' in row and pd.notna(row.get('state_id')) and state_centroid is not None:
            st = row['state_id']
            if st in state_centroid.index:
                row['latitude'] = state_centroid.at[st,'latitude']
                row['longitude'] = state_centroid.at[st,'longitude']
                row['_imputed_by'] = f'state:{st}'
                return row

        if global_mean is not None:
            row['latitude'] = global_mean['latitude']
            row['longitude'] = global_mean['longitude']
            row['_imputed_by'] = 'global_mean'
        return row

    # Apply imputation only when needed
    if total_unmatched > 0:
        unmatched_before = unmatched.shape[0]
        unmatched['_imputed_by'] = np.nan
        unmatched = unmatched.apply(impute_row, axis=1)
        walmart_with_coords.loc[unmatched.index, ['latitude','longitude','_imputed_by']] = unmatched[['latitude','longitude','_imputed_by']]

        remaining_unmatched = walmart_with_coords['latitude'].isna().sum()
        print(f'Remaining unmatched rows after imputation: {remaining_unmatched}')
        print('\nImputation method counts (top examples):')
        if '_imputed_by' in walmart_with_coords.columns:
            print(walmart_with_coords['_imputed_by'].value_counts().head(20))

        out_file = 'walmart_with_coordinates_imputed.csv'
        walmart_with_coords.to_csv(out_file, index=False)
        print(f'Imputed dataset saved to: {out_file}')

walmart_with_coords columns: ['index', 'SHIPPING_LOCATION', 'DEPARTMENT', 'CATEGORY', 'SUBCATEGORY', 'BREADCRUMBS', 'SKU', 'PRODUCT_URL', 'PRODUCT_NAME', 'BRAND', 'PRICE_RETAIL', 'PRICE_CURRENT', 'PRODUCT_SIZE', 'tid', 'latitude', 'longitude', 'city', 'state_id']
Total unmatched rows: 8446

Top 30 unmatched SHIPPING_LOCATION values (with counts):
SHIPPING_LOCATION
6010    8446
Name: count, dtype: int64
Number of prefixes available for prefix-based imputation: 896
Number of states available for state-based imputation: 56
Remaining unmatched rows after imputation: 0

Imputation method counts (top examples):
_imputed_by
prefix:601    8446
Name: count, dtype: int64
Remaining unmatched rows after imputation: 0

Imputation method counts (top examples):
_imputed_by
prefix:601    8446
Name: count, dtype: int64
Imputed dataset saved to: walmart_with_coordinates_imputed.csv
Imputed dataset saved to: walmart_with_coordinates_imputed.csv
